In [106]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
from matplotlib import pyplot
from scipy import stats
import numpy as np
import h5py
import pandas as pd
import tensorflow as tf

In [107]:

f = h5py.File('Z:\gait_database\LSTM_data_onlyMarkersEMG.h5', 'r')

In [108]:

print(list(f.keys()))

['x_test', 'x_train', 'x_valid', 'y_test', 'y_train', 'y_valid']


In [109]:
x_test = f['x_test'][:,:,-16:]
x_train = f['x_train'][:,:,-16:]
x_valid = f['x_valid'][:,:,-16:]
y_test = f['y_test']
y_train = f['y_train']
y_valid = f['y_valid']


In [110]:
print("print x_train =",x_train.shape, "y_train =",y_train.shape)
print("print x_test = ", x_test.shape,"y_test  =", y_test.shape)
print("print x_valid =",x_valid.shape, "y_valid =",y_valid.shape)

print x_train = (301, 425, 16) y_train = (301, 1)
print x_test =  (100, 425, 16) y_test  = (100, 1)
print x_valid = (100, 425, 16) y_valid = (100, 1)


In [111]:
import pandas as pd

# normalizacja markerów


In [112]:
stats.zscore(x_train, axis=1, ddof=1)
stats.zscore(x_test, axis=1, ddof=1)


array([[[-1.25140077, -1.52061236, -0.9568416 , ..., -1.17357742,
         -0.7553479 , -0.99558973],
        [-1.18720482, -1.47145736, -0.92632328, ..., -1.14645063,
         -0.73268414, -0.96597235],
        [-1.12483156, -1.42329691, -0.89548411, ..., -1.11927373,
         -0.71086637, -0.93716674],
        ...,
        [-0.80622575, -0.72916412, -0.83278146, ..., -1.02397797,
         -0.16617194,  0.29518572],
        [-0.80663657, -0.72917669, -0.83259032, ..., -1.02396978,
         -0.16604347,  0.29570355],
        [-0.80677831, -0.72918305, -0.83252476, ..., -1.02396754,
         -0.16599945,  0.29586955]],

       [[-1.1698649 , -1.19795081, -1.07978408, ..., -1.10784437,
         -1.09899674, -0.86633166],
        [-1.13991174, -1.15675782, -1.03208723, ..., -1.08644203,
         -1.04669873, -0.81784853],
        [-1.11075199, -1.11720054, -0.98644515, ..., -1.06524242,
         -0.99536321, -0.76692116],
        ...,
        [-0.89199105, -0.54784614, -0.60918289, ..., -

In [113]:
def load_group(filenames):
    loaded = list()
    for name in filenames:
        data = f
        loaded.append(data)
    # grupę stosów, tak aby funkcje były trzecim wymiarem
    loaded = dstack(loaded)
    return loaded

In [114]:
    c = list(zip(x_train,y_train ))
    np.random.shuffle(c)
    d = list(zip(x_test,y_test ))
    np.random.shuffle(d)
    v = list(zip(c,d))
    np.random.shuffle(v)

In [115]:
def load_dataset():
    
    c = list(zip(x_train,y_train ))
    np.random.shuffle(c)
    d = list(zip(x_test,y_test ))
    np.random.shuffle(d)
    v = list(zip(c,d))
    np.random.shuffle(v)
    
    TrainX = x_train
    TestX = x_test
    TrainY = y_train
    TestY = y_test

    TrainY = to_categorical(TrainY)
    TestY = to_categorical(TestY)
    #print(trenujX.shape, trenujY.shape, testX.shape, testY.shape)
    return TrainX, TrainY, TestX, TestY

In [120]:
def evaluate_model(TrainX, TrainY, TestX, TestY):
    verbose, epochs, batch_size = 0, 10, 64 # 15, 64 domyslnie, dla 10, 64 działa najlepiej verbose pokaze liczbe epok
    
    n_timesteps, n_features, n_outputs = TrainX.shape[1], TrainX.shape[2], TrainY.shape[1]
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.1)) # 0.5
    
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # fit network
    model.fit(TrainX, TrainY, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(TestX, TestY, batch_size=batch_size, verbose=1)
    return accuracy

In [121]:
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [122]:
def run_experiment(repeats=10):
    # load data
    TrainX, TrainY, TestX, TestY = load_dataset()
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(TrainX, TrainY, TestX, TestY)
        score = score * 100.0
        print('>#%d: %.3f' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [123]:
run_experiment(repeats=10)

2/2 [==============================] - 0s 34ms/step - loss: 0.6847 - accuracy: 0.6100
>#1: 61.000
2/2 [==============================] - 0s 41ms/step - loss: 1.6333 - accuracy: 0.6000
>#2: 60.000
2/2 [==============================] - 0s 35ms/step - loss: 1.3425 - accuracy: 0.5200
>#3: 52.000
2/2 [==============================] - 0s 47ms/step - loss: 1.3762 - accuracy: 0.5300
>#4: 53.000
2/2 [==============================] - 0s 37ms/step - loss: 1.4798 - accuracy: 0.5000
>#5: 50.000
2/2 [==============================] - 0s 64ms/step - loss: 0.8433 - accuracy: 0.5300
>#6: 53.000
2/2 [==============================] - 0s 57ms/step - loss: 0.7478 - accuracy: 0.7100
>#7: 71.000
2/2 [==============================] - 0s 69ms/step - loss: 0.9285 - accuracy: 0.4900
>#8: 49.000
2/2 [==============================] - 0s 65ms/step - loss: 1.0120 - accuracy: 0.5000
>#9: 50.000


2/2 [==============================] - 0s 62ms/step - loss: 1.4354 - accuracy: 0.5500
>#10: 55.000
[61.000001430511475, 60.00000238418579, 51.99999809265137, 52.99999713897705, 50.0, 52.99999713897705, 70.99999785423279, 49.000000953674316, 50.0, 55.000001192092896]
Accuracy: 55.400% (+/-6.468)
